In [1]:
import os
import selenium.webdriver
import csv
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options  # for suppressing the browser
from concurrent.futures import ThreadPoolExecutor, wait

In [2]:
url1= "https://walmart.com/ip"

url_sets=["https://www.walmart.com/browse/tv-video/all-tvs/3944_1060825_447913",
    "https://www.walmart.com/browse/computers/desktop-computers/3944_3951_132982",
         "https://www.walmart.com/browse/electronics/all-laptop-computers/3944_3951_1089430_132960",
         "https://www.walmart.com/browse/prepaid-phones/1105910_4527935_1072335",
         "https://www.walmart.com/browse/electronics/portable-audio/3944_96469",
         "https://www.walmart.com/browse/electronics/gps-navigation/3944_538883/",
         "https://www.walmart.com/browse/electronics/sound-bars/3944_77622_8375901_1230415_1107398",
         "https://www.walmart.com/browse/electronics/digital-slr-cameras/3944_133277_1096663",
         "https://www.walmart.com/browse/electronics/ipad-tablets/3944_1078524"]

categories=["TVs","Desktops","Laptops","Prepaid_phones","Audio","GPS","soundbars","cameras","tablets"]
#codelist= ["7FCB6T54IGW4","4TENT7IJPA5E","38R56ZV9ISK5","6OQGB0M3MO7C","20CTD1EQDDV2"]

In [3]:
def run_process(page_number,url_sets):
    for i in range(len(url_sets)):
        top_n= ["1","2","3","4","5","6","7","8","9","10"]
        url_cat=url_sets[i]+"?page="+top_n[page_number]
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        driver= webdriver.Chrome(executable_path='C:/chromedriver.exe',options=option)
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])

In [4]:
futures = []
final_results = []
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process,number,url_sets)
        )
wait(futures)

DoneAndNotDoneFutures(done={<Future at 0x170b2416240 state=finished returned NoneType>, <Future at 0x170afc52278 state=finished returned NoneType>, <Future at 0x170b2422470 state=finished returned NoneType>, <Future at 0x170b24166a0 state=finished returned NoneType>, <Future at 0x170b2416eb8 state=finished returned NoneType>, <Future at 0x170b24164e0 state=finished returned NoneType>, <Future at 0x170b2416b38 state=finished returned NoneType>, <Future at 0x170b2422198 state=finished returned NoneType>, <Future at 0x170b24169b0 state=finished returned NoneType>, <Future at 0x170b24227f0 state=finished returned NoneType>}, not_done=set())

In [5]:
len(final_results)

2557

top_n= ["1","2","3","4","5","6","7","8","9","10"]
final_results = []
for i in range(len(top_n)):
    url_cat="https://www.walmart.com/browse/electronics/all-laptop-computers/3944_3951_1089430_132960"+"?page="+top_n[i]
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver= webdriver.Chrome(executable_path='C:/chromedriver.exe',options=option)
    driver.get(url_cat)
    body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
    driver.quit()
    soupBody_cat = BeautifulSoup(body_cat)
    for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

        final_results.append(tmp['data-id'])

In [6]:
len(final_results)

codelist=final_results

In [7]:
codelist[0]

'35X5YDKAGYGK'

In [8]:
def run_process2(i,codelist):
    #creating a list without the place taken in the first loop
    print(i)
    item_wlmt=codelist[i]
    url2=url1+"/"+item_wlmt
    print(url2)
    option = webdriver.ChromeOptions()
    option.add_argument('headless')

    try:
        driver= webdriver.Chrome(executable_path='C:/chromedriver.exe',options=option) # Chrome driver is being used.
        print ("Requesting URL: " + url2)
        driver.get(url2)   # URL requested in browser.
        print ("Webpage found ...")
        time.sleep(5)
        # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        print("Closing Chrome ...") # No more usage needed.
        driver.quit() 				# Browser Closed.

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
        #print(soupBody)

        h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
        divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
        liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
        liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
        spanProductPrice = soupBody.find("span", {"class": "price-group"})
        spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
        spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
        ################# exceptions #########################
        if divProductDesc is None:
            divProductDesc="Not Available"
        else:
            divProductDesc=divProductDesc
                
        if liProductBreadcrumb_parent is None:
            liProductBreadcrumb_parent="Not Available"
        else:
            liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
        if liProductBreadcrumb_active is None:
            liProductBreadcrumb_active="Not Available"
        else:
            liProductBreadcrumb_active=liProductBreadcrumb_active
                
        if spanProductPrice is None:
            spanProductPrice="NA"
        else:
            spanProductPrice=spanProductPrice

        if spanProductRating is None or spanProductRating_count is None:
            spanProductRating=0.0
            spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
        else:
            spanProductRating=spanProductRating.text
            spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
        reco_prods=[]
        for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
            reco_prods.append(tmp['data-product-id'])

        if len(reco_prods)==0:
            reco_prods=["Not available"]
        else:
            reco_prods=reco_prods
        print(len(reco_prods))
        WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                         liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                         spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])
    except Exception as e:
        print (str(e))

In [9]:
futures2 = []
WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
with ThreadPoolExecutor() as executor:
    for number in range(len(codelist)):
        futures2.append(
            executor.submit(run_process2,number,codelist)
        )
wait(futures2)

0
https://walmart.com/ip/35X5YDKAGYGK
1
https://walmart.com/ip/6EPON5SWVN42
2
https://walmart.com/ip/4655HRO97D87
3
https://walmart.com/ip/76IYDXY6KXI9
4
https://walmart.com/ip/7K3V50QQP6XT5
https://walmart.com/ip/17QIVAFFTDZM

6
https://walmart.com/ip/280435SHDJ4Z
7
https://walmart.com/ip/7FM9X732CQ8N
8
https://walmart.com/ip/3K0M9NY5IC45
9
https://walmart.com/ip/4UEBVQURA5YF
10
https://walmart.com/ip/4W0KP0GRD7MO11

https://walmart.com/ip/2RNJNKOV86PH
12
https://walmart.com/ip/74N4QUGMS0I1
13
https://walmart.com/ip/1Z53ODZKMZC4
14
https://walmart.com/ip/2RKSQC7C8QKZ
15
16
https://walmart.com/ip/63IDMM7042YQ
https://walmart.com/ip/2X3MQ9SM84AA
17
https://walmart.com/ip/5M597D73BLL2
18
https://walmart.com/ip/3TBPGUH5BC9G
19
https://walmart.com/ip/3I9EXG7JUILG
20
https://walmart.com/ip/66IMXADRGEEH
21
https://walmart.com/ip/4U07UQJGM4NA
22
https://walmart.com/ip/1U9W6CPANKC1
23
https://walmart.com/ip/45UCO8DZT1O9
24
https://walmart.com/ip/5ZAPHYRJFPWO
25
https://walmart.com/ip/2SHA6OIWA

Webpage found ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Webpage found ...
Closing Chrome ...
25
67
https://walmart.com/ip/7IRBT68DZFI9
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...Closing Chrome ...

Closing Chrome ...
20
68
https://walmart.com/ip/1R6FGKIIRWRX
Closing Chrome ...
Getting data from DOM ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/6PO39JHKB9C4
Closing Chrome ...
Getting data from DOM ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/7IRBT68DZFI9
15
69
https://walmart.com/ip/5D81OMDWSQYW
Requesting URL: https://walmart.com/ip/4N6E3JK9ZHXH
Getting data from DOM ...
Closing Chrome ...
Closing Chrome ...
Webpage found ...
Requesting URL: https://walmart.com/ip/1R6FGKIIRWRX
Webpage found ...
Requesting URL: https://walmart.com/ip/4COB5TK59N3S
Getting data 

Closing Chrome ...
Requesting URL: https://walmart.com/ip/7FD9Y0AMQ9BA
25
106
https://walmart.com/ip/6YIT7D32TGVH
Getting data from DOM ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/228AKZIC1I9M
Webpage found ...
Webpage found ...
Webpage found ...Requesting URL: https://walmart.com/ip/2R0PUN4NAI88Webpage found ...Webpage found ...



Getting data from DOM ...
15
107
https://walmart.com/ip/7AGDBBJTUY4M
Webpage found ...
1
'NoneType' object has no attribute 'text'
108
https://walmart.com/ip/6RAPB0KEBJEZ
Webpage found ...
Getting data from DOM ...
Closing Chrome ...
Webpage found ...
Webpage found ...
30
109
https://walmart.com/ip/5FMO33R8L4H5
Requesting URL: https://walmart.com/ip/6YIT7D32TGVH
15
110
https://walmart.com/ip/6LXIB9703WQS
14
111
https://walmart.com/ip/3TZTWV9WWB5H
25
112
https://walmart.com/ip/3MLAIZ2MJVE8
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
Requesting URL: https://walmart.com/ip/6RAPB0KEBJEZ
25
113
https://walmart.com/ip/5GWM16X

Closing Chrome ...
Getting data from DOM ...
Getting data from DOM ...
Requesting URL: https://walmart.com/ip/70JL5PBWIRCE
Getting data from DOM ...
5
156
https://walmart.com/ip/1V9Y8DB8U9AH
Webpage found ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/2GWEA1DW7H1M
Getting data from DOM ...
Closing Chrome ...
1
'NoneType' object has no attribute 'text'
157
https://walmart.com/ip/5HF3T8I5XH2A
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/3OT6FQ51HYFN
Requesting URL: https://walmart.com/ip/530DS38JD2CB
25Webpage found ...Closing Chrome ...


158
https://walmart.com/ip/3BXK5Y6ID6G0
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...
25Getting data from DOM ...
159
https://walmart.com/ip/15K9NMI1EYUF

1
'NoneType' object has no attribute 'text'
160
https://walmart.com/ip/3SINC39BP3DJ
Getting data from DOM ...
Requesting URL: https://walmart.com/ip/1V9Y8DB8U9AH
Getting data from DOM ...
Requesting URL: https:

Requesting URL: https://walmart.com/ip/79GFYOCX7ZJ7
Requesting URL: https://walmart.com/ip/4655HRO97D87
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
10
207
https://walmart.com/ip/2MS8A0E27C03
Requesting URL: https://walmart.com/ip/401LE67KRAFP
Webpage found ...Requesting URL: https://walmart.com/ip/6KRX4TE3RLR9

Getting data from DOM ...
Closing Chrome ...
Getting data from DOM ...
Requesting URL: https://walmart.com/ip/59CNCY5BVXOG
25
208
https://walmart.com/ip/68C67P1R0JLT
20
209
https://walmart.com/ip/6L7KXWJFHRNQ
Requesting URL: https://walmart.com/ip/2MS8A0E27C03
Getting data from DOM ...
Webpage found ...
20
210
https://walmart.com/ip/46VQULKOD37S
Webpage found ...Closing Chrome ...

Requesting URL: https://walmart.com/ip/68C67P1R0JLT
24
211
https://walmart.com/ip/4NRR2ARN1P0N
Requesting URL: https://walmart.com/ip/6L7KXWJFHRNQ
Webpage found ...
Closing Chrome ...
Getting data from DOM ...
Webpage found ...
30
212
https://walmart.com/ip/58GFYJJAJA40
Requesting U

...

Getting data from DOM ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...Requesting URL: https://walmart.com/ip/7491E4SKR0KH

Getting data from DOM ...
Webpage found ...
Getting data from DOM ...
1
2404
https://walmart.com/ip/2XTYASMKF36E
5
2405
https://walmart.com/ip/7EMEP1UNSX1H
Requesting URL: https://walmart.com/ip/2UNZP2MZQI6O
Requesting URL: https://walmart.com/ip/5J8IKM2NN8W1
Getting data from DOM ...Getting data from DOM ...
Getting data from DOM ...
Getting data from DOM ...

Getting data from DOM ...
15
2406
https://walmart.com/ip/225457508M5R
Webpage found ...
Requesting URL: https://walmart.com/ip/1F9DU4WF9KPD
1
'NoneType' object has no attribute 'text'
2407
https://walmart.com/ip/5K8IE7J9KIEF
1
'NoneType' object has no attribute 'text'15
2408
https://walmart.com/ip/705EHTCJMZT1

2409
https://walmart.com/ip/5GHG3Z30ERU8
Requesting URL: https://walmart.com/ip/2XTYASMKF36E
Requesting URL: https://walmart

Requesting URL: https://walmart.com/ip/5LTLT91GSABP
Closing Chrome ...
11
2452
https://walmart.com/ip/4VL72D9UNUQO

2453
https://walmart.com/ip/492HWDIM8GSP
10
2454
https://walmart.com/ip/3RW2HM2QYDFX
Closing Chrome ...
Requesting URL: https://walmart.com/ip/2V5R80PTP8DF
1Requesting URL: https://walmart.com/ip/4ORTUGPPQB3K

2455
https://walmart.com/ip/2M6L9RPKMWVZ
Closing Chrome ...
Requesting URL: https://walmart.com/ip/2KR50LQOE2X0
Closing Chrome ...
Getting data from DOM ...
Getting data from DOM ...
1
2456
https://walmart.com/ip/53VF1YUC1FAF
Requesting URL: https://walmart.com/ip/4VL72D9UNUQORequesting URL: https://walmart.com/ip/492HWDIM8GSP

Webpage found ...
Closing Chrome ...Getting data from DOM ...

Requesting URL: https://walmart.com/ip/3RW2HM2QYDFX
Webpage found ...
Closing Chrome ...15
2457
https://walmart.com/ip/1T0I6WP572J5

Requesting URL: https://walmart.com/ip/2M6L9RPKMWVZ
Getting data from DOM ...
1015
15
24582459
https://walmart.com/ip/32CRCZZ9SQ2R

2460

https://wa

Webpage found ...
Webpage found ...
Webpage found ...
Requesting URL: https://walmart.com/ip/1XKF4A6H90WH
Webpage found ...
Closing Chrome ...
Webpage found ...
Requesting URL: https://walmart.com/ip/6JA5MDJA5GM9
Requesting URL: https://walmart.com/ip/1G91BJDMQ4EO
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/3HYXOLV6ASWJ
Getting data from DOM ...
1
2504
https://walmart.com/ip/1AVIN0QX0AFF
Webpage found ...
20
250510Closing Chrome ...

https://walmart.com/ip/66LPENOON25H
2506

https://walmart.com/ip/5076JTVFL69K
Getting data from DOM ...
Getting data from DOM ...
Closing Chrome ...25
1
2507
'NoneType' object has no attribute 'text'

2508https://walmart.com/ip/5I3K412KYNK5

https://walmart.com/ip/7ESTP09K4T2A
Closing Chrome ...
Closing Chrome ...
15
Webpage found ...
Closing Chrome ...
2509
https://walmart.com/ip/1JCSH7BDGG00
1
2510
https://walmart.com/ip/78BT48Y2Z05E
Closing Chrome ...
Getting data from 

Getting data from DOM ...
1
2550
https://walmart.com/ip/3SNA90SSYX6G
1
'NoneType' object has no attribute 'text'
2551
https://walmart.com/ip/75L7GQ4EK1HI
1
2552
https://walmart.com/ip/3228PS39L547
25
2553
https://walmart.com/ip/2KPFX90LUYJH
1
'NoneType' object has no attribute 'text'
2554
https://walmart.com/ip/796ARWY0SOPX
Getting data from DOM ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/0YYD7LI56CP6
Closing Chrome ...
Closing Chrome ...
Closing Chrome ...
Getting data from DOM ...
Closing Chrome ...
Requesting URL: https://walmart.com/ip/75L7GQ4EK1HIRequesting URL: https://walmart.com/ip/38GYTUJW7CQV

Requesting URL: https://walmart.com/ip/3SNA90SSYX6G
1
2555
https://walmart.com/ip/62IWO2UXRKG7
Getting data from DOM ...
20
2556
https://walmart.com/ip/6LMSIJ3OTOY1
Requesting URL: https://walmart.com/ip/2KPFX90LUYJH
Requesting URL: https://walmart.com/ip/3228PS39L547
1
'NoneType' object has no attribute 'text'
Requesting URL: https://walmart.com/ip/796ARWY0SOPX
Closin

DoneAndNotDoneFutures(done={<Future at 0x170bb9c0048 state=finished returned NoneType>, <Future at 0x170baa20080 state=finished returned NoneType>, <Future at 0x170baa200b8 state=finished returned NoneType>, <Future at 0x170b2c800b8 state=finished returned NoneType>, <Future at 0x170b2c80128 state=finished returned NoneType>, <Future at 0x170bb9c0128 state=finished returned NoneType>, <Future at 0x170b2c80160 state=finished returned NoneType>, <Future at 0x170baa20160 state=finished returned NoneType>, <Future at 0x170bb9c0198 state=finished returned NoneType>, <Future at 0x170bb9c01d0 state=finished returned NoneType>, <Future at 0x170b2c801d0 state=finished returned NoneType>, <Future at 0x170bb9c0208 state=finished returned NoneType>, <Future at 0x170b2c80240 state=finished returned NoneType>, <Future at 0x170baa20278 state=finished returned NoneType>, <Future at 0x170bb9c0278 state=finished returned NoneType>, <Future at 0x170c9360278 state=finished returned NoneType>, <Future at 0

In [ ]:
for pg in range(len(url_sets)):
    # number of pages per category
    top_n= ["1","2","3","4","5","6","7","8","9","10"]
    url_category=url_sets[pg]
    print("Category:",categories[pg])
    final_results = []
    for i_1 in range(len(top_n)):
        print("Page number within category:",i_1)
        url_cat=url_category+"?page="+top_n[i_1]
        driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe')
        driver.get(url_cat)
        body_cat = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        driver.quit()
        soupBody_cat = BeautifulSoup(body_cat)
        
        
        for tmp in soupBody_cat.find_all('div', {'class':'search-result-gridview-item-wrapper'}):

            final_results.append(tmp['data-id'])
    codelist=final_results
    print("Total number of prods:",len(codelist))
    # base URL for product page
    url1= "https://walmart.com/ip"

    # Data Headers
    WLMTData = [["Product_code","Product_name", "Product_description","Product_URL",
                 "Breadcrumb_parent","Breadcrumb_active",
                "Product_price", "Rating_Value","Rating_Count","Recommended_Prods"]]
    for i in range(len(codelist)):
        #creating a list without the place taken in the first loop
        print(i)
        item_wlmt=codelist[i]
        url2=url1+"/"+item_wlmt
        #print(url2)
        
        try:
            driver= webdriver.Chrome(executable_path='C:/Drivers/chromedriver.exe') # Chrome driver is being used.
            print ("Requesting URL: " + url2)

            driver.get(url2)   # URL requested in browser.
            print ("Webpage found ...")
            time.sleep(5)
            # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
            body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
            print("Closing Chrome ...") # No more usage needed.
            driver.quit() 				# Browser Closed.

            print("Getting data from DOM ...")
            soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup
            #print(soupBody)

            h1ProductName = soupBody.find("h1", {"class": "prod-ProductTitle prod-productTitle-buyBox font-bold"})
            divProductDesc = soupBody.find("div", {"class": "about-desc about-product-description xs-margin-top"})
            liProductBreadcrumb_parent = soupBody.find("li", {"data-automation-id": "breadcrumb-item-0"})
            liProductBreadcrumb_active = soupBody.find("li", {"class": "breadcrumb active"})
            spanProductPrice = soupBody.find("span", {"class": "price-group"})
            spanProductRating = soupBody.find("span", {"itemprop": "ratingValue"})
            spanProductRating_count = soupBody.find("span", {"class": "stars-reviews-count-node"})
            
            ################# exceptions #########################
            if divProductDesc is None:
                divProductDesc="Not Available"
            else:
                divProductDesc=divProductDesc
                
            if liProductBreadcrumb_parent is None:
                liProductBreadcrumb_parent="Not Available"
            else:
                liProductBreadcrumb_parent=liProductBreadcrumb_parent
                
            if liProductBreadcrumb_active is None:
                liProductBreadcrumb_active="Not Available"
            else:
                liProductBreadcrumb_active=liProductBreadcrumb_active
                
            if spanProductPrice is None:
                spanProductPrice="NA"
            else:
                spanProductPrice=spanProductPrice

            if spanProductRating is None or spanProductRating_count is None:
                spanProductRating=0.0
                spanProductRating_count="0 ratings"
            #print(spanProductBreadcrumb_active.text)
            else:
                spanProductRating=spanProductRating.text
                spanProductRating_count=spanProductRating_count.text


            ### Recommended Products
            reco_prods=[]
            for tmp in soupBody.find_all('a', {'class':'tile-link-overlay u-focusTile'}):
                reco_prods.append(tmp['data-product-id'])

            if len(reco_prods)==0:
                reco_prods=["Not available"]
            else:
                reco_prods=reco_prods
            print(len(reco_prods))
            WLMTData.append([codelist[i],h1ProductName.text, divProductDesc.text,url2,
                            liProductBreadcrumb_parent.text,liProductBreadcrumb_active.text,
                            spanProductPrice.text,spanProductRating,spanProductRating_count,reco_prods])




            except Exception as e:
                print (str(e))

#WLMTData        

In [10]:
df=pd.DataFrame(WLMTData)
df.columns = df.iloc[0]
df=df.drop(df.index[0])
df

,Product_code,Product_name,Product_description,Product_URL,Breadcrumb_parent,Breadcrumb_active,Product_price,Rating_Value,Rating_Count,Recommended_Prods
1,35X5YDKAGYGK,"RCA 50"" Class 4K Ultra HD (2160P) LED TV (RLDE...","The RCA 50"" 2160p Class 4K Ultra High-Definiti...",https://walmart.com/ip/35X5YDKAGYGK,Electronics,/All TVs,$219.99,3.9,1254 ratings,"[5YZBEUJO11MO, 1CFXZI1QNUKO, 6SA8LLNNSLMP, 51O..."
2,2RKSQC7C8QKZ,"RCA 55"" Class 4K Ultra HD (2160P) HDR Roku Sma...",Enjoying 4K Ultra HD just got easier. RCA Roku...,https://walmart.com/ip/2RKSQC7C8QKZ,Electronics,/All TVs,$279.99,4.2,595 ratings,"[1CFXZI1QNUKO, 2X3MQ9SM84AA, 6SA8LLNNSLMP, 5ZA..."
3,4U07UQJGM4NA,"LG 65"" Class 4K UHD 2160P Smart TV 65UN7300PUF...",Empower entertainment with LG UHD TV. LG's UHD...,https://walmart.com/ip/4U07UQJGM4NA,Electronics,/All TVs,$646.99,4.3,154 ratings,"[2MJRWZI33VET, 46VQULKOD37S, 2Z0G09Y8QKOL, 3ML..."
4,75BV5KCCV6DE,"SAMSUNG 70"" Class 4K Crystal UHD (2160P) LED S...",This smart TV unlocks hidden detail at four ti...,https://walmart.com/ip/75BV5KCCV6DE,Electronics,/All TVs,$947.00,4.4,2843 ratings,"[46VQULKOD37S, 5NGD72QFL38K, 3YFRMHEUUEZD, 3BX..."
5,7K3V50QQP6XT,"TCL 32"" Class 1080P FHD LED Roku Smart TV 3 Se...",The 3-Series TCL Roku TV puts all your enterta...,https://walmart.com/ip/7K3V50QQP6XT,Electronics,/All TVs,$149.99,4.4,208 ratings,"[2Z3IRIL1ATIA, 60R8ZDOQCUAS, 5HF3T8I5XH2A, 2J9..."
...,...,...,...,...,...,...,...,...,...,...
2002,2GR4W5AW89OG,"Vankyo MatrixPad Z1 Kids 7 inch, 32GB ROM, Kid...",Vankyo MatrixPad Z1 Kids Made for fun Designed...,https://walmart.com/ip/2GR4W5AW89OG,Electronics,/Android Tablets for Kids,$69.99,4.6,16 ratings,"[23G4WOJHC0S8, 5RWM6EUJ1XMV, 6B670NL5Q1D1, 1P9..."
2003,6LMSIJ3OTOY1,Cell Phone Holder Phone Stand for Desk Tablet ...,,https://walmart.com/ip/6LMSIJ3OTOY1,Cell Phones,/All Phone Car Holders,$10.19,4.4,10 ratings,[Not available]
2004,3228PS39L547,Targus VersaVu Rotating Case for 9.7-inch iPad...,Whether you?re bringing your iPad to the offic...,https://walmart.com/ip/3228PS39L547,Electronics,"/iPad Cases, Sleeves & Bags",$14.66,4.8,12 ratings,"[1094OQH0XWCG, 17HS4ZS6T8UI, 1R69MYZQ4TD6, 6YV..."
2005,62IWO2UXRKG7,Apple USB Home Travel Adapter/ Lightning Cable...,Apple USB Home Travel Charger Adapter/ Lightni...,https://walmart.com/ip/62IWO2UXRKG7,Cell Phones,/Apple Phone Chargers,$12.97,4.1,76 ratings,"[4CFBRDTTCHG0, 2UYU75A4276H, 64MDH00JKSBS, 4OS..."


In [15]:
# Output File for FlightsData. This file will have the data in comma separated form.
df.to_csv('electronics_out.csv',index=False)